<a href="https://colab.research.google.com/github/jafetimbre/optimus/blob/master/src/proj/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install dask[dataframe] requests aiohttp sodapy -q 

In [70]:
import pandas as pd

import dask 
import dask.dataframe as dd
import dask.bag as db
from dask.distributed import Client, progress

from sodapy import Socrata

In [79]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2021-03-16 20:11:16--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.193.233.154, 52.203.96.188, 54.84.220.80, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.193.233.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  34.1MB/s    in 0.4s    

2021-03-16 20:11:17 (34.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [82]:
dask_client = Client(n_workers=4, threads_per_worker=1)
dask_client

/usr/local/lib/python3.7/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: tcp://127.0.0.1:40267 Dashboard: http://127.0.0.1:35933/status,Cluster Workers: 4 Cores: 4 Memory: 27.31 GB


In [83]:
get_ipython().system_raw('./ngrok http 8787 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://17bf46002971.ngrok.io


In [84]:
socrata_client = Socrata("data.cityofchicago.org", None)
socrata_results = socrata_client.get("85ca-t3if", limit=20000)

In [100]:
pandas_df = pd.DataFrame.from_records(socrata_results, exclude=['location'], coerce_float=True).convert_dtypes(convert_string=False)
pandas_df.head()

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,report_type,crash_type,hit_and_run_i,damage,date_police_notified,prim_contributory_cause,sec_contributory_cause,street_no,street_direction,street_name,beat_of_occurrence,num_units,most_severe_injury,injuries_total,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,injuries_unknown,crash_hour,crash_day_of_week,crash_month,latitude,longitude,intersection_related_i,statements_taken_i,crash_date_est_i,private_property_i,work_zone_i,photos_taken_i,dooring_i,work_zone_type,workers_present_i,lane_cnt,rd_no
0,905e9c290e6c6decb7624e8f3347392b728307adc62bc3...,2021-03-16T00:03:00.000,30,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,WET,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,Y,"OVER $1,500",2021-03-16T00:09:00.000,FOLLOWING TOO CLOSELY,NOT APPLICABLE,1300,E,70TH ST,321,2,NO INDICATION OF INJURY,0,0,0,0,0,4,0,0,3,3,41.76792296,-87.593990135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,034634c5bd01075fd8d12ebd9dc5a0e359b4e2ba767087...,2021-03-15T23:39:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,WET,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,NaN,"OVER $1,500",2021-03-15T23:42:00.000,UNABLE TO DETERMINE,NOT APPLICABLE,1334,W,NORTH AVE,1433,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,23,2,3,41.910820508,-87.661844691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c70cb93a520a2f3bdd621566069de5b08bb37a72f3fcc9...,2021-03-15T21:41:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DARKNESS,FIXED OBJECT,NOT DIVIDED,STRAIGHT AND LEVEL,WET,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,NaN,$500 OR LESS,2021-03-15T21:58:00.000,UNABLE TO DETERMINE,UNABLE TO DETERMINE,318,E,71ST ST,322,1,FATAL,1,1,0,0,0,0,0,21,2,3,41.765772529,-87.617633917,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,290733e1291972bc680b476ba537f8e80483dca72b1914...,2021-03-15T21:20:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,SNOW OR SLUSH,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,Y,"OVER $1,500",2021-03-15T21:50:00.000,NOT APPLICABLE,NOT APPLICABLE,3604,W,GRAND AVE,1112,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,21,2,3,41.902473674,-87.716817465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c6b2c047f3a479e62dcab861c368af61e61811971e87ff...,2021-03-15T21:20:00.000,30,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,WET,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,N,"OVER $1,500",2021-03-15T21:50:00.000,FAILING TO YIELD RIGHT-OF-WAY,CELL PHONE USE OTHER THAN TEXTING,4607,W,FULLERTON AVE,2522,2,NO INDICATION OF INJURY,0,0,0,0,0,2,0,21,2,3,41.924186061,-87.742116995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
pandas_df.dtypes

crash_record_id                  object
crash_date                       object
posted_speed_limit               object
traffic_control_device           object
device_condition                 object
weather_condition                object
lighting_condition               object
first_crash_type                 object
trafficway_type                  object
alignment                        object
roadway_surface_cond             object
road_defect                      object
report_type                      object
crash_type                       object
hit_and_run_i                    object
damage                           object
date_police_notified             object
prim_contributory_cause          object
sec_contributory_cause           object
street_no                        object
street_direction                 object
street_name                      object
beat_of_occurrence               object
num_units                        object
most_severe_injury               object


In [93]:
data = dd.from_pandas(pandas_df, npartitions=10)

In [94]:
data.head()

distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/distributed/protocol/core.py", line 114, in loads
    header = msgpack.loads(header, use_list=False, **msgpack_opts)
  File "msgpack/_unpacker.pyx", line 195, in msgpack._cmsgpack.unpackb
ValueError: tuple is not allowed for map key
distributed.core - ERROR - tuple is not allowed for map key
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/distributed/core.py", line 386, in handle_stream
    msgs = yield comm.read()
  File "/usr/local/lib/python3.7/dist-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/usr/local/lib/python3.7/dist-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/usr/local/lib/python3.7/dist-packages/distributed/comm/tcp.py", line 207, in read
    deserializers=deserializers)
  File "/usr/local/lib/python3.7/dist-packages/t

CancelledError: ignored